# A primer on decorators in python

## 0. Some background on functions in python
A function in python (or any other language) returns a value based on the input arguments.

In [ ]:
def add_one(a: int) -> float:
    return a + 1

add_one(15)

Sometimes they return nothing, and just perform some action. For example, the `print` function just logs stuff to the console, but returns nothing. For the purposes of this tutorial, we will focus on functions that return outputs.

Functions are also objects, and can be passed in as arguments into other functions

In [ ]:
def operate(func, x):
    result = func(x)
    return result

operate(add_one, 5)

Functions can also be returned from other functions. In the cell below, we can see that the function `is_returned` is returned from the `is_called` function. You can see the difference with `sth` and `sth()`

In [ ]:
def is_called():
    def is_returned():
        print('Hi')
    return is_returned

sth = is_called()
print(sth)
sth()

## 1. What is a decorator?
A decorator is a "wrapper" for a function. It takes in a function, adds some functionality to it, and then returns the a modified version of the function. We will see a few examples of decorators in the next few cells

In [ ]:
def make_pretty(func):
    def inner():
        print("I got decorated")
        func()
    return inner


def ordinary():
    print("I am ordinary")

In [ ]:
ordinary()

In [ ]:
pretty = make_pretty(ordinary)
pretty()

We see that the `make_pretty` function added some functionality to the `ordinary` function. `make_pretty` is a **decorator**, and it decorates the `ordinary` function. Notice that the nature of the `ordinary` function does not change - it is merely extended. We can use the `@` symbol along with the name of the decorator function and place it above the definition of the function to be decorated. For example:

In [ ]:
@make_pretty
def ordinary():
    print("I am ordinary")

In [ ]:
ordinary()

This is the same thing as `ordinary = make_pretty(ordinary)`. The `@` symbol here is just [**syntactic sugar**](https://en.wikipedia.org/wiki/Syntactic_sugar) to implement decorators. 

We will see a few examples of simple decorators

In [ ]:
def do_twice(func):
    """ Will perform the function twice"""
    def inner():
        func()
        func()
    return inner

def say_hello():
    print('Hello!!!!')
    
greeting = do_twice(say_hello)
greeting()

Now let's use the decorator properly:

In [ ]:
@do_twice
def say_hello():
    print('Hello!!!!')
    
say_hello()

Now let's try to pass an argument to the `greet_me` function and see what happens

In [ ]:
@do_twice
def greet_me(name):
    print(f'Hi {name}, happy to meet you today')

greet_me('Joel')

We get an error because the `inner` function doesn't take any arguments. To fix this, we include positional and keyword arguments in the definition of the `inner` function as such:

In [ ]:
def do_twice(func):
    def inner(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return inner

@do_twice
def greet_me(name):
    print(f'Hi {name}, happy to meet you today')

@do_twice
def say_location(name, location):
    print(f'Hi {name}, you live in {location}')
    
greet_me('Bemsi')
say_location('Joel', location = 'Leeds')

We can also implement a decorator that takes in an argument. For example, with the `@do_twice` decorator, we might want to change it to `do_n_times`, and then pass in the number of times we want the inner function to run as an argument. 

In [ ]:
def do_n_times(n_times=2):
    def wrapper(func):
        def inner(*args, **kwargs):
            for i in range(n_times):
                func(*args, **kwargs)
        return inner
    return wrapper

@do_n_times(5)
def greet_me(name):
    print(f'Hi {name}, happy to meet you today')
    
greet_me('Bemsi')

To understand what is going on, we look at the decorator from the innermost function, `inner`. It takes arbitrary arguments and returns the value of the decorated function, `func`, but also does this `n` times. The `n_times` argument is seemingly not used in the `wrapper` function, but creates a closure so that the value of `n_times` is stored until it will be used by the `inner` function.

In [ ]:
@do_n_times()
def say_location(name, location):
    print(f'Hi {name}, you live in {location}')
    
say_location('Bemsi', 'Cambridge')

Pay attention to the fact that we used brackets in the decorator here - that's because the `do_n_times` now takes an argument. We used decorators without brackets previously because the decorators didn't take any arguments

# 2. Some use cases for decorators
Anytime you want to extend the functionality of function. By using decorators, you can easily generate different permutations of functionality that you want instead of creating a large number of objects -making your code complex and bloated. For example analytics, logging, validation, runtime checks, etc. We will look at a few examples

## 2.1 Analytics and logging
Logging is an important part of software engineering. In python, decorators can enable us log quite easily

In [ ]:
from datetime import datetime
import logging

def log_datetime(func):
    '''Log the date and time of a function'''

    def wrapper(*arg):
        print(f'Function: {func.__name__}\nRun on: {datetime.today().strftime("%Y-%m-%d %H:%M:%S")}')
        print(f'{"-"*30}')
        func(*arg)
    return wrapper

def time_dec(func):
    def wrapper(*arg):
        t = time.time()
        res = func(*arg)
        print(f"{func.__name__} took {time.time()-t} to complete")
        return res
    return wrapper

@log_datetime
@time_dec
def multiply(a,b):
    return a*b

In [ ]:
multiply(5,6)

We see two things in the above example:
    
- We can chain decorators together. We chained `log_datetime` and `time_dec` together, and saw what the result is.
- The `log_datetime` function name got lost in the first decorator, and we instead saw `wrapper` as the function name. This can pose problems when debugging decorated functions. As we have seen so far in this tutorial, decorators wrap functions. The original function name, its docstring and parameter list are all hidden by the wrapper closure. In `multiply` above for example, we see the `wrapper` closure's metadata, and this can be a problem when debugging. 

To solve this issues, we use the `functools.wraps` decorator in the standard Python library. This decorator copies the metadata from the undecorated function to the decorated closure. We see this in the next example:


In [ ]:
from functools import wraps

def log_datetime(func):
    '''Log the date and time of a function'''
    
    @wraps(func)
    def wrapper(*arg):
        print(f'Function: {func.__name__}\nRun on: {datetime.today().strftime("%Y-%m-%d %H:%M:%S")}')
        print(f'{"-"*30}')
        func(*arg)
    return wrapper

def time_dec(func):
    @wraps(func)
    def wrapper(*arg):
        t = time.time()
        res = func(*arg)
        print(f"{func.__name__} took {time.time()-t}s to complete")
        return res
    return wrapper

In [ ]:
@log_datetime
@time_dec
def multiply(a,b):
    return a*b

In [ ]:
multiply(5,6)

We now see appropriate closures. This logic can be used to measure other aspects of the performance of a function - speed, memory usage, etc.

## 2.2 Validation 
Imagine that we have a set of functions whose arguments must be positive. If any argument is negative, that’s an error. Here is a decorator that raises a `ValueError` if that happens:

In [ ]:
from functools import wraps
import math

def check_positive(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        if any([arg <0 for arg in args]):
            raise ValueError(f"{func.__name__} accepts only positive arguments. Please check your inputs.")
        res = func(*args, **kwargs)
        return res
    return wrapper

@check_positive
def geometric_average(a,b):
    return math.sqrt(a*b)

@check_positive
def arithmetic_average(a,b):
    return (a+b)/2

@check_positive
def logarithm(a, b):
    return math.log(a) - math.log(b)

print(logarithm(5,6))
print(arithmetic_average(5,2))
print(geometric_average(5,-6))


The above code would be easier to write and maintain than writing:

In [ ]:
def geometric_average(x,y):
    if x <0 or y <0:
        raise ValueError(f"All arguments must be positive for function geometric average to work")
    return math.sqrt(a*b)
    
def arithmetic_average(x,y):
    if x <0 or y <0:
        raise ValueError(f"All arguments must be positive for function geometric average to work")
    return (a+b)/2

def logarithm(x,y):
    if x <0 or y <0:
        raise ValueError(f"All arguments must be positive for function geometric average to work")
    return math.log(y) - math.log(x)

In general it would be better to have:
```@conditional_function 
def fun(): 
...
```
than multiple instances of 
```
def fun():   
    if conditional_function ...
```


## 2.3 Decorators with classes
You can decorate the methods of a class (you are probably already doing so). The most commonly used decorators here are `@classmethod`, `@staticmethod` and `@property`. The `@classmethod` and `@staticmethod` decorators are used to define methods inside a class namespace that are not connected to a particular instance of that class. The `@property` decorator is used to customize getters and setters for class attributes. 

In [ ]:
class Rectangle:
    def __init__(self, length, width):
        self._length = length
        self._width = width
    
    @property
    def length(self):
        return self._length
    
    @length.setter
    def length(self, value):
        if value >= 0:
            self._length = value
        else:
            raise ValueError("Length must be positive")
    
    @property
    def area(self):
        return self._length * self._width
    
    @classmethod
    def square(cls, length):
        return cls(_length, _length)
    
    @staticmethod
    def e():
        return 2.712

In [ ]:
r = Rectangle(5,2)

In [ ]:
s = Rectangle.square(5)
print(f's is a square with length {s._length} and width {s._width}, and its area is {s.area}')

In [ ]:
print(Rectangle.e())

In the `Rectangle` class:
 - `length` is a mutable property - it can be set to a different value. When we define a setter method, we can do some validation to make sure that the right values are passed onto it
 - `.area` is an immutable property - properties without `.setter()` methods cannot be changed
 - `.square` is a class method - it is not bound to one particular instance of `Rectangle`
 - `.e()` is a static method. It doesn't really dependent on the `Rectangle` class, except for being a part of its namespace

We can also decorate entire classes in the same fashion as we decorated functions - that is, decorating the entire class. A good example of this is the [dataclass](https://docs.python.org/3/library/dataclasses.html) module.

In [ ]:
from dataclasses import dataclass

@dataclass
class Rectangle:
    length: float
    width: float
    
    @property
    def area(self) -> float:
        return self.length * self.width
    
r = Rectangle(5.4, 3.6)
print(r.area)

## 2.4 Retry decorators
Sometimes you have a function and want to be able to catch if a particular call gets raised when it's run. For example, you are making requests to a server, and sometimes you get a server error - you need to retry that request. Imagine further that you have many of such functions making such calls - you may end up having boilerplate code. But you can use a decorator for this

In [ ]:
def retry(func):
    @wraps(func)
    def retried_func(*args, **kwargs):
        MAX_TRIES = 5
        tries = 0
        while True:
            resp = func(*args, **kwargs)
            if resp.status_code == 500 and tries < MAX_TRIES:
                tries += 1
                continue
            break
        return resp
    return retried_func

@retry
def make_api_call():
    pass

You may have noticed that we can refactor the above code so that we can pass in the `max_tries` as an argument to the `retry` decorator.

In [ ]:
def retry(max_tries=3):
    @wraps(func)
    def retry_decorator(func):
        def retried_func(*args, **kwargs):
            tries = 0
            while True:
                resp = func(*args, **kwargs)
                if resp.status_code == 500 and tries < max_tries:
                    tries += 1
                    continue
                break
            return resp
        return retried_func
    return retry_decorator

@retry(5)
def make_api_call():
    pass

# Learn more:

You can read more about decorators here:
 - [Real Python's Primer on decorators](https://realpython.com/instance-class-and-static-methods-demystified/)
 - [O Reilly - 5 reasons you need to learn to write Python decorators](https://www.oreilly.com/content/5-reasons-you-need-to-learn-to-write-python-decorators/)
 - [FreeCodeCamp - Python Decorators](https://www.freecodecamp.org/news/python-decorators-explained-with-examples/)


---

***This is a living document, so feel free to suggest any modifications!***